## Deliverable 2. Create a Customer Travel Destinations Map.

In [42]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [43]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kaitangata,NZ,2022-05-22 23:47:32,-46.2817,169.8464,50.14,70,97,16.51,overcast clouds
1,1,Bandarbeyla,SO,2022-05-22 23:45:19,9.4942,50.8122,81.27,78,82,16.60,broken clouds
2,2,Haines Junction,CA,2022-05-22 23:47:33,60.7522,-137.5108,60.75,21,100,10.36,overcast clouds
3,3,Castro,BR,2022-05-22 23:47:33,-24.7911,-50.0119,49.14,81,28,2.57,scattered clouds
4,4,Marawi,PH,2022-05-22 23:47:33,7.9986,124.2928,71.51,90,100,1.66,overcast clouds


In [44]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [45]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Bandarbeyla,SO,2022-05-22 23:45:19,9.4942,50.8122,81.27,78,82,16.60,broken clouds
10,10,Kahului,US,2022-05-22 23:45:13,20.8947,-156.4700,87.82,40,0,18.41,clear sky
12,12,Butaritari,KI,2022-05-22 23:45:36,3.0707,172.7902,82.15,78,53,12.62,broken clouds
16,16,Vaini,TO,2022-05-22 23:45:23,-21.2000,-175.2000,82.56,83,40,8.05,scattered clouds
20,20,Cockburn Town,TC,2022-05-22 23:47:38,21.4612,-71.1419,80.56,69,40,17.27,scattered clouds
...,...,...,...,...,...,...,...,...,...,...,...
639,639,Forbesganj,IN,2022-05-23 00:07:01,26.3000,87.2500,77.61,76,96,6.33,overcast clouds
662,662,Noumea,NC,2022-05-23 00:08:11,-22.2763,166.4572,79.00,99,30,13.38,scattered clouds
665,665,The Valley,AI,2022-05-23 00:08:13,18.2170,-63.0578,80.82,75,14,16.58,few clouds
669,669,Huainan,CN,2022-05-23 00:08:14,32.6264,116.9969,75.94,59,6,6.76,clear sky


In [46]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                189
City                   189
Country                189
Date                   189
Lat                    189
Lng                    189
Max Temp               189
Humidity               189
Cloudiness             189
Wind Speed             189
Current Description    189
dtype: int64

In [47]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#### no empty rows

In [48]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bandarbeyla,SO,81.27,broken clouds,9.4942,50.8122,
10,Kahului,US,87.82,clear sky,20.8947,-156.4700,
12,Butaritari,KI,82.15,broken clouds,3.0707,172.7902,
16,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,
20,Cockburn Town,TC,80.56,scattered clouds,21.4612,-71.1419,
21,Atuona,PF,77.92,light rain,-9.8000,-139.0333,
26,Rikitea,PF,75.51,light rain,-23.1203,-134.9692,
29,Hithadhoo,MV,83.77,overcast clouds,-0.6000,73.0833,
30,Tuy Hoa,VN,80.04,overcast clouds,13.0833,109.3000,
35,Hobyo,SO,80.08,overcast clouds,5.3505,48.5268,


In [49]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [67]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df_clean = hotel_df.copy()
hotel_df_clean.dropna(inplace=True)
hotel_df_clean

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Bandarbeyla,SO,81.27,broken clouds,9.4942,50.8122,JABIR HOTEL
10,Kahului,US,87.82,clear sky,20.8947,-156.4700,Maui Seaside Hotel
12,Butaritari,KI,82.15,broken clouds,3.0707,172.7902,Isles Sunset Lodge
16,Vaini,TO,82.56,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
20,Cockburn Town,TC,80.56,scattered clouds,21.4612,-71.1419,Osprey Beach Hotel
...,...,...,...,...,...,...,...
639,Forbesganj,IN,77.61,overcast clouds,26.3000,87.2500,होटल
662,Noumea,NC,79.00,scattered clouds,-22.2763,166.4572,Hôtel Le Lagon
665,The Valley,AI,80.82,few clouds,18.2170,-63.0578,CeBlue Villas & Beach Resort
669,Huainan,CN,75.94,clear sky,32.6264,116.9969,GME Huainan Tianjiaan District Chaoyang East R...


In [68]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
hotel_df_clean.to_csv(output_data_file, index_label="City_ID")

In [82]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd> and <dd>{Max Temp} °F</dd></dl>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df_clean.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df_clean[["Lat", "Lng"]]

In [85]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df_clean[["Lat", "Lng"]]
max_temp = hotel_df_clean["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))